In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

from time import sleep
import gc

# 1. 시가총액 TOP 300 크롤링

In [10]:
selenium.__version__

'3.8.0'

### Scrap web

In [11]:
soups = []

for i in range(3):
    # 1. Strart Web-driver
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("start-maximized")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(options=chrome_options, executable_path=r'C:\Users\dltnd\AppData\Roaming\chromedriver_win32\chromedriver.exe')

    # 2. Scrap web 
    url = "https://coinmarketcap.com/ko/?page="+str(i+1)
    driver.get(url)

    height = driver.execute_script('return document.body.scrollHeight')
    for timer in range(1000,height,1000):
        driver.execute_script("window.scrollTo(0, "+str(timer)+")") 
        sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    soups.append(soup)
    
    driver.quit()

### Extract Coin Name, code, Market Cap

In [12]:
def simplify_caps(caps):
    new_caps = []
    cho, uk, cheon = 10**12, 10**8, 10**3
    for cap in caps:
        if cap>cho:
            tmp = round(cap/cho,1)
            tmp = str(tmp)+'Jo'
        elif cap>uk:
            tmp = int(cap/uk)
            tmp = str(tmp)+'Uk'
        else:
            tmp = int(cap/cheon)
            tmp = str(tmp)+'Ch'
        new_caps.append(tmp)
    return new_caps

dfs = []
for soup in soups:
    tbody = soup.tbody

    names = [x.get_text() for x in tbody.find_all("p",{'class':'sc-1eb5slv-0 iworPT'})]
    codes = [x.get_text() for x in tbody.find_all("p",{'class':'sc-1eb5slv-0 gGIpIK coin-item-symbol'})]
    caps = [x.get_text() for x in soup.find_all("span",{'class':'sc-1ow4cwt-1 ieFnWP'})]

    assert len(names)==100, "Not 100 names" 
    assert len(codes)==100, "Not 100 codes" 
    assert len(caps)==100, "Not 100 caps" 

    # Market cap 숫자 및 단위 변환
    caps = [int(cap[1:].replace(',','')) for cap in caps]
    simple_caps = simplify_caps(caps)

    tmp_df = pd.DataFrame({'name':names, 'code':codes, 'cap':caps, 'simple_cap':simple_caps})
    dfs.append(tmp_df)

df = pd.concat(dfs, ignore_index=True)

#del dfs, soups
#gc.collect()

# 2. 실시간 "거래대금 순위변동" 크롤링 
# + 텔레그램 알림

### by python-binance API

In [13]:
from binance.client import Client
from datetime import datetime
import time
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import telegram
from tabulate import tabulate
import dataframe_image as dfi

In [14]:
with open("api.txt") as f:
    lines = f.readlines()
    api_key = lines[0].strip()
    api_secret  = lines[1].strip()
    
client = Client(api_key=api_key, api_secret=api_secret)

In [15]:
def generate_rank(x, rank_lst):
    curr20 = rank_lst[-1][:rank_n]
    plot_dict = {sym:[] for sym in curr20}
    for sym in curr20:
        for rank in rank_lst:
            idx = rank.index(sym)
            plot_dict[sym].append(idx+1)
    return plot_dict

def rank_matplotlib(x, plot_dict):
    print('x:',x)
    print('plot_dict:',plot_dict)
    fig, ax = plt.subplots(figsize=(18, 5), constrained_layout=True)
    for k, v in plot_dict.items(): 
        ax.plot(x, v, label=k, marker='o')  
    ax.set_xlabel('x label')  
    ax.set_ylabel('y label') 
    ax.set_title("Simple Plot")  
    ax.legend()
    ax.set_xticks(x, x)
    plt.gca().invert_yaxis()
    plt.show()

def rank_plotly(x, plot_dict):
    df = pd.DataFrame(plot_dict, index=x)

    fig = go.Figure()
    
    for col in df.columns:
        fig.add_trace(go.Scatter(x=df.index, y=df[col], text=df[col],
                        mode='lines+markers+text', # 'lines' or 'markers'
                        name=col))
        
    fig.update_traces(marker={'size':12},
                      textposition='middle center')
    fig.update_layout(
        height=850,
        yaxis = {'autorange':'reversed'},
        title = '거래대금 순위변동 Top '+str(rank_n)
    )
   
    fig.show()
    fig.write_image('img/fig.png')
    
def simplify_dollars(dollars):
    new_dolls = []
    M = 10**6
    for dollar in dollars:
        if dollar>M:
            tmp = round(dollar/M,2)
            tmp = str(tmp)+'M'
        new_dolls.append(tmp)
    return new_dolls

In [17]:
# Telegram bot 생성
chat_id = # Enter your telegram bot ID
token = # Enter your telegram bot token
bot = telegram.Bot(token=token)

period = 3600 # 순위변동 감지 간격(s)
rank_n = 30 # rank Top k
x, rank_lst = [], []
display_cols = ['symbol','24h_change','simple_24hVolume','simple_cap']
past = datetime.now()

while True:
    now = datetime.now()
    print(now)
    
    # 1시간 후 순위변동 그래프 생성 및 전송
    diff = now-past
    if diff.seconds >= period: 
        past = now
        # list 갯수 제한
        if len(x)>=16:
            del x[0], rank_lst[0]
        x.append(now)
        rank_lst.append(usdt_markets.symbol.to_list())
        plot_dict = generate_rank(x, rank_lst)
        
        # 그래프 그리기
        rank_plotly(x, plot_dict)
        
        # 텔래그램 그래프 전송
        bot.send_photo(chat_id, photo=open('img/fig.png', 'rb'))

    markets = client.get_all_tickers()

    # Select USDT spot
    name, symbol_lst = 'USDT', []
    for market in markets:
        symbol =  market['symbol']
        if name in symbol and symbol.find(name)!=0:
            symbol_lst.append(symbol)

    # Request current market info.(20s)       
    usdt_markets = pd.DataFrame()
    for symbol in symbol_lst:
        tmp_dict = client.get_ticker(symbol=symbol)
        tmp_dict['24h_volume'] = float(tmp_dict['weightedAvgPrice'])*float(tmp_dict['volume'])
        usdt_markets = usdt_markets.append(tmp_dict, ignore_index=True)
    
    usdt_markets = usdt_markets[list(tmp_dict.keys())]
    
    # 24h volume 단순화
    usdt_markets['simple_24hVolume'] = simplify_dollars(usdt_markets['24h_volume'].to_list())
    
    # 거래대금 순으로 정렬
    usdt_markets = usdt_markets.sort_values(by=['24h_volume'], ascending=False)
    usdt_markets = usdt_markets.reset_index(drop=True)
    
    # 24h price change 칼럼 추가
    lastPrice = np.array([float(x) for x in usdt_markets['lastPrice'].to_list()])
    openPrice = np.array([float(x) for x in usdt_markets['openPrice'].to_list()])
    usdt_markets['24h_change'] = (lastPrice-openPrice)/openPrice*100
    
    # 시가총액 칼럼 합치기
    usdt_markets['code'] = [x.replace('USDT','') for x in usdt_markets['symbol'].to_list()]
    usdt_markets = pd.merge(usdt_markets, df[['code','simple_cap']], on='code')
    
    # 최종 데이터프레임 이미지 저장
    dfi.export(usdt_markets[:30][display_cols], 'img/usdt_markets.png')
    
    # 텔레그램 데이터프레임 전송
    bot.send_photo(chat_id, photo=open('img/usdt_markets.png', 'rb'))
    #msg = tabulate(usdt_markets[:rank_n][['symbol','simple_cap']], headers='keys', tablefmt='psql')
    #bot.send_message(chat_id, msg)
    
    time.sleep(60)

2022-03-01 14:09:34.256342


C:\Users\dltnd\.venv\colab\lib\site-packages\ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in true_divide


2022-03-01 14:10:58.123794
2022-03-01 14:12:19.450308
2022-03-01 14:13:40.962308
2022-03-01 14:15:02.093854


KeyboardInterrupt: 

In [61]:
print(tabulate(usdt_markets[:30][['symbol','24h_change','simple_24hVolume','simple_cap']], headers='keys', tablefmt='psql'))

+----+------------+--------------+--------------------+--------------+
|    | symbol     |   24h_change | simple_24hVolume   | simple_cap   |
|----+------------+--------------+--------------------+--------------|
|  0 | BTCUSDT    |    0.600577  | 1252.0M            | 952.2조      |
|  1 | ETHUSDT    |    1.4644    | 1004.09M           | 410.4조      |
|  2 | BUSDUSDT   |    0.010005  | 472.6M             | 21.1조       |
|  3 | SHIBUSDT   |   -3.25463   | 367.77M            | 19.2조       |
|  4 | XRPUSDT    |   -1.48284   | 280.19M            | 45.4조       |
|  5 | GALAUSDT   |   -3.02618   | 236.53M            | 2.4조        |
|  6 | DOGEUSDT   |   -3.87139   | 214.75M            | 23.0조       |
|  7 | ONGUSDT    |   44.7745    | 195.07M            | 3077억       |
|  8 | BNBUSDT    |    0.0497018 | 192.65M            | 77.9조       |
|  9 | SOLUSDT    |    2.2241    | 161.37M            | 35.6조       |
| 10 | LUNAUSDT   |    2.81315   | 160.98M            | 25.0조       |
| 11 | SANDUSDT  